`03-pdf-rag.ipynb`

In [1]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()


True

In [ ]:
llm.invoke('testing')

In [ ]:
# 1. 문서 로딩(Loader)
from langchain_community.document_loaders import PyMuPDFLoader

# 2. 문서 나누기(Split)
from langchain_text_splitters import RecursiveCharacterTextSplitter

# 3. 문서 변환하기(Embed) => GPT를 사용하기 때문에 OpenAI 에서 제공하는 Embedding 사용
from langchain_openai import OpenAIEmbeddings

# 4. Embedding 저장하기 (Vector Store)
from langchain_community.vectorstores import FAISS

In [ ]:
%pip install pymupdf

In [ ]:
# 1. Load
loader = PyMuPDFLoader('./data.pdf')
docs = loader.load()
print(f'문서 페이지 수: {len(docs)}')

In [ ]:
# 2. Split
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
split_docs = text_splitter.split_documents(docs)
print(f'분할된 청크의 수: {len(split_docs)}')

In [ ]:
# 3. Embed Setting
embeddings = OpenAIEmbeddings()

In [ ]:
%pip install faiss-cpu

In [ ]:
# 4. Vectorstore
vectorstore = FAISS.from_documents(documents=split_docs, embedding=embeddings)

In [ ]:
# 5. 검색(retrieve) => vectorstore 내장 검색기
# 6. 프롬프트 생성(prompt)
from langchain_core.prompts import PromptTemplate

# 7. LLM 
from langchain_openai import ChatOpenAI

# 8. Chain

In [ ]:
# 5. Retrieve
retriever = vectorstore.as_retriever()

In [ ]:
results = retriever.invoke('구글 투자액')
for r in results:
    print(r.page_content)
    print()

In [34]:
# 6. Prompting
text = '''
넌 질문-답변을 도와주는 AI 비서야.
아래 제공되는 Context를 통해서 사용자 Question에 대해 답을 해줘야해.

모르는 내용이 질문으로 들어오면 `몰라요ㅠㅠ` 라고 답을 해줘.
답은 한국어로 만들어 줘. 
Context에는 없어도, 추론하거나 계산할 수 있는 답변은 최대한 만들어 봐.

# Question:
{question}

# Context:
{context}


# Answer:
'''

prompt = PromptTemplate.from_template(text)

In [43]:
# 7. LLM
llm = ChatOpenAI(model="gpt-3.5-turbo-1106", temperature=0.8)

In [44]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()
# 8. Chain
chain = (
    {'context': retriever, 'question': RunnablePassthrough()}
    | prompt
    | llm
    | parser
)

In [45]:
chain.invoke('구글 투자액의 총합은 얼마야? 여러 문서들을 돌면서 검색해서 더해')

'몰라요ㅠㅠ'